In [130]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

import pandas as pd
import category_encoders as ce
import sklearn.model_selection


TypeError: __init__() takes 1 positional argument but 2 were given

Import and Parse Data

In [121]:
train_data = pd.read_csv('../data/LOANS_TRAIN.csv')
test_data = pd.read_csv('../data/LOANS_TEST.csv')

train_X = train_data.iloc[:,:-1]
train_y = train_data.iloc[:,-1]
test_X = test_data.iloc[:,:]
test_y = test_data.iloc[:,0]

#Remove redundant data
# TODO: decide on which data to keep and which to drop
train_data = train_data.drop(columns=['id', 'emp_title', 'issue_d', 'addr_state', 'purpose', 'initial_list_status', 'title', 'application_type', 'grade'])
test_data = test_data.drop(columns=['id', 'emp_title', 'issue_d', 'addr_state', 'purpose', 'initial_list_status', 'title', 'application_type', 'grade'])

#Strip percent(%) from int_rate
train_data['int_rate'] = train_data['int_rate'].str.rstrip('%').astype(float)
test_data['int_rate'] = test_data['int_rate'].str.rstrip('%').astype(float)

#Strip percent(%) from revol_util
train_data['revol_util'] = train_data['revol_util'].str.rstrip('%').astype(float)
test_data['revol_util'] = test_data['revol_util'].str.rstrip('%').astype(float)

#TODO: Select which data should be encoded
# ce_OHE = ce.OneHotEncoder(cols=['loan_status', 'application_type', 'home_ownership', 'verification_status'])
# train_data = ce_OHE.fit_transform(train_data)

loan_status = {'Fully Paid':0, 'Charged Off':1}
train_data['loan_status'].replace(loan_status, inplace=True)

#Replace subgrade with numerical value
ce_OHE = ce.OneHotEncoder(cols=['home_ownership', 'verification_status', 'sub_grade', 'zip_code', 'emp_length', 'mort_acc', 'earliest_cr_line'])
train_data = ce_OHE.fit_transform(train_data)

# train_X.is_null().sum()

train_X = train_data.iloc[:,:-1]
train_y = train_data.iloc[:,-1]
test_X = test_data.iloc[:,:]

train_X, val_X, train_y, val_y = sklearn.model_selection.train_test_split(train_X, train_y, test_size=0.2, random_state=0)

print(train_X)

        loan_amnt  term_(months)  int_rate  installment  sub_grade_1  \
89545        9800             36     19.52       361.82            0   
190443      16000             60     18.49       410.58            0   
57541       13275             36      6.62       407.60            0   
124679       3000             36     16.29       105.91            0   
119027       5000             36     12.35       166.91            0   
...           ...            ...       ...          ...          ...   
152315      24175             36     12.12       804.35            0   
176963      35000             36     15.80      1227.05            0   
117952       2400             36     14.33        82.42            0   
173685       7750             36     12.12       257.86            0   
43567       12175             36     10.99       398.54            1   

        sub_grade_2  sub_grade_3  sub_grade_4  sub_grade_5  sub_grade_6  ...  \
89545             0            0            0          

Neural Network

In [133]:
torch.autograd.detect_anomaly()

model = nn.Sequential(
    nn.Flatten(),
    nn.BatchNorm1d(1572),
    nn.Linear(1572, 900),
    nn.BatchNorm1d(900),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(900, 600),
    nn.BatchNorm1d(600),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(600, 300),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(300, 100),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(100, 10),
)

/var/folders/np/bctbg3vd7bq8kh3pf7hhzb940000gn/T/ipykernel_14610/3156475876.py:1: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  torch.autograd.detect_anomaly()


In [123]:
train_y = pd.DataFrame({'loan_status':train_y.values})
val_y = pd.DataFrame({'loan_status':val_y.values})
print(type(val_y))

<class 'pandas.core.frame.DataFrame'>


In [136]:
import torch.optim as optim

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [125]:

import torch.utils.data as data_utils

train_y = torch.tensor(train_y['loan_status'].values.astype(np.longlong))
train_X = torch.tensor(train_X.values.astype(np.float32)) 
train_tensor = data_utils.TensorDataset(train_X, train_y) 
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = 128, shuffle = True)

val_y = torch.tensor(val_y['loan_status'].values.astype(np.longlong))
val_X = torch.tensor(val_X.values.astype(np.float32)) 
test_tensor = data_utils.TensorDataset(val_X, val_y) 
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = 32, shuffle = True)

# train_y = torch.from_numpy(train_y)
# train_X = torch.from_numpy(train_X)
# train_tensor = data_utils.TensorDataset(train_X, train_y) 
# train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = 64, shuffle = True)

# val_y = torch.from_numpy(val_y)
# val_X = torch.from_numpy(val_X)
# val_tensor = data_utils.TensorDataset(val_X, val_y) 
# val_loader = data_utils.DataLoader(dataset = val_tensor, batch_size = 32, shuffle = True)

# train_X = torch.tensor(train_X)
# train_y = torch.IntTensor(train_y)


# val_X = torch.tensor(val_X)
# val_y = torch.IntTensor(val_y)


# train = data_utils.TensorDataset(train_X, train_y)
# train_loader = data_utils.DataLoader(train, batch_size=64, shuffle=True)

# test = data_utils.TensorDataset(val_X, val_y)
# test_loader = data_utils.DataLoader(test, batch_size=32, shuffle=True)

# train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True) 

In [137]:
# Some layers, such as Dropout, behave differently during training
model.train()
for epoch in range(10):
    for batch_idx, ((data), target) in enumerate(train_loader):
        # Erase accumulated gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(data)

        # Calculate loss
        loss = loss_fn(output, target)

        # Backward pass
        loss.backward()
        
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)

        # Weight update
        optimizer.step()

    # Track loss each epoch
    print('Train Epoch: %d  Loss: %.4f' % (epoch + 1,  loss.item()))

Train Epoch: 1  Loss: nan


KeyboardInterrupt: 